iris dataset을 활용한 회귀 모델 구현
feature 3개 : sepal_length, sepal_width, petal_length
target 1개 : petal_width

In [127]:
# 모듈 로딩
import torch                    # tensor, 일반 function
import torch.nn as nn           # nn관련 class, function
import torch.nn.functional as F   # nn관련 AL, LF, MF
import pandas as pd

- 데이터 로딩

In [128]:
DataPath = r"C:\Users\KDP-23\Desktop\VSCode\LocalData\DL\data\iris.csv"
dataDF = pd.read_csv(DataPath,usecols = [0,1,2,3])
dataDF

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [129]:
## 피쳐 타겟 분리

featureDF = dataDF.loc[:, :'petal.length']
featureDF

,sepal.length,sepal.width,petal.length
0,5.1,3.5,1.4
1,4.9,3.0,1.4
2,4.7,3.2,1.3
3,4.6,3.1,1.5
4,5.0,3.6,1.4
...,...,...,...
145,6.7,3.0,5.2
146,6.3,2.5,5.0
147,6.5,3.0,5.2
148,6.2,3.4,5.4


In [130]:
# 피쳐 DF => ndarray => Tensor
xTensor = torch.FloatTensor(featureDF.values)

In [131]:
## 타겟 분리

targetDF = dataDF.loc[:, ['petal.length']]
targetDF

,petal.length
0,1.4
1,1.4
2,1.3
3,1.5
4,1.4
...,...
145,5.2
146,5.0
147,5.2
148,5.4


In [132]:
# 타겟 DF => ndarray => Tensor
yTensor = torch.FloatTensor(targetDF.values)

In [133]:
## 모델 인스턴스 생성
mModel = nn.Sequential(
    nn.Linear(3,10),
    nn.ReLU(),
    nn.Linear(10,30),
    nn.ReLU(),
    nn.Linear(30,1))

- 학습 => 순방향(forward()메서드) + 역방향(손실함수. backward()메서드)
    - 손실함수 LF : MSE, MAE, RMSE, ...
    - 층별 W,b 업데이트 최적화 인스턴스 : SGD, Adam, ...

In [134]:
import torch.optim as optim  # W,b를 경사하강법에 따라 업데이트 관련 모듈

In [135]:
# 최적화 인스턴스 생성
adam = optim.Adam(mModel.parameters())

# 손실함수 
mse = nn.MSELoss()

In [136]:
# 순방향 학습 forward()
# 입력 : shape(150,3)
outputTensor = mModel(xTensor)
outputTensor.shape, yTensor.shape

(torch.Size([150, 1]), torch.Size([150, 1]))

In [137]:
# 순반향 합습 후 오차/잔차 계산
((outputTensor - yTensor)**2).mean()

tensor(18.0157, grad_fn=<MeanBackward0>)

In [138]:
loss = mse(outputTensor, yTensor)

In [139]:
# 역전파 : loss를 최소화 하기 위해서 업데이트 진행
loss.backward()     # 현재의 W에서 - (loss미분계산값 * lr), b - (loss미분계산값 * lr)
adam.zero_grad()    # 이전에 저장된 값을 삭제 > 하지않으면 W,b 누적
adam.step()         # 모델안 층별 W, b를 새롭게 업데이트

- - - 이때까지는 한번 학습한것 - - -   
이제 for 문으로 계속 학습

In [140]:
# 학습 진행 => 지정된 횟수 만큼 학습
epochs = 10

for epoch in range(epochs):
    
    # 순방향
    outputTensor = mModel(xTensor)

    # 오차값 
    loss = mse(outputTensor, yTensor)

    # 업데이트
    loss.backward()     
    adam.zero_grad()    
    adam.step()

    # 학습 모니터링
    print(f'{epoch}/{epochs} loss : {loss}')

0/10 loss : 18.015676498413086
1/10 loss : 18.015676498413086
2/10 loss : 18.015676498413086
3/10 loss : 18.015676498413086
4/10 loss : 18.015676498413086
5/10 loss : 18.015676498413086
6/10 loss : 18.015676498413086
7/10 loss : 18.015676498413086
8/10 loss : 18.015676498413086
9/10 loss : 18.015676498413086
